In [7]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

### Following
- https://realpython.com/python-keras-text-classification/

In [8]:
df = pd.read_csv('posts_all_filtered.csv', header=None)
df.columns = ['category','content']
print(len(df.index))

463


In [9]:
train_x, valid_x, train_y, valid_y = train_test_split(df['content'], df['category'])


In [10]:
vectorizer = CountVectorizer()
vectorizer.fit(train_x)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [ ]:
vectorizer.vocabulary_

In [11]:
train_X = vectorizer.transform(train_x)
valid_X  = vectorizer.transform(valid_x)

In [12]:
valid_X

<116x7135 sparse matrix of type '<class 'numpy.int64'>'
	with 11843 stored elements in Compressed Sparse Row format>

In [13]:
classifier = LogisticRegression()
classifier.fit(train_X, train_y)

/Users/juan/Documents/manu/dev/reddit-ml/venv/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/juan/Documents/manu/dev/reddit-ml/venv/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [14]:
score = classifier.score(valid_X, valid_y)

In [15]:
print("Accuracy:", score)

Accuracy: 0.7241379310344828


In [16]:
valid_x.head(12).reset_index()

,index,content
0,78,To protect the names and identities of everyon...
1,455,My girlfriend (20F) and I (26M) broke up after...
2,215,This series was posted around either 2006-2007...
3,461,My fiancée and I have known each other since h...
4,108,I burnout and lose motivation every now and then.
5,60,I've been in a serious relationship for over 2...
6,374,"So, my ex and I dated for 4 years. We planned ..."
7,286,"Study uses fake financial firms, swaps headsho..."
8,226,This came up in another thread and was suggest...
9,429,"After over 18 years of regular churchgoing, I ..."


In [17]:
valid_y.head(12).reset_index()

,index,category
0,78,relationships
1,455,relationships
2,215,tipofmytongue
3,461,relationships
4,108,Entrepreneur
5,60,relationships
6,374,CasualConversation
7,286,science
8,226,tipofmytongue
9,429,CasualConversation


In [18]:
classifier.predict(valid_X[5])

array(['relationships'], dtype=object)